#**BBM409: Machine Learning Laboratory**
==================================

**Programming Assignment-2**

**Instructors:** 
*   Ahmet Burak Can
*   Burçak Asal

**Prepared By:** 
* Mert Tazeoğlu `(21946606)`

**Problem Definition:** 

In this assignment, we will use Decision Tree (ID3) Algorithm in order to handle classification problem.

#**Part-1: Employee Attrition Prediction**

##**1.1 - Importing Required Libraries**
*   **`Numpy:`** To perform a wide variety of mathematical operations on datasets.
*   **`Pandas:`** To analyse and manipulate tabular data in different dataframes.
*   **`Math:`** To make basic mathematical and statistical operations on dataframes.
*   **`Operator:`** To make critical dictionary operations in an easier way.
*   **`OS:`** To delete file content if file exists (to prevent writing file without deleting).

In [1]:
import pandas as pd
import numpy as np
import math
import operator
import os

##**1.2 - Implementing Required Functions**

Explanations for each of implemented functions are at below (with order):

*   **`Cross Validation Splitter:`** This function divides dataframe into k folds and returns one fold as validation dataset and others as training dataset. In each turn, index of validation dataset changes.

*   **`Continious Feature Discretizer:`** This function discretizes continous features with equal width (or distance interval) binning. This method bins to partition the range of the variable into k equal-width intervals.

*   **`Record Groupper:`** This function groups discretized features by their values in a dictionary. This is a critical helper function for calculation of both of system and feature entropies.

*   **`System Entropy Calculator:`** This function calculates total entropy of the system. This is a critical helper function for calculation of information gain and feature selection.

*   **`Feature Entropy Calculator:`** This function calculates total entropy of the feature. This is a critical helper function for calculation of information gain and feature selection.

*   **`Gain Calculator:`** This function calculates total information gains with using entropy functions for each of the features. This is a critical helper function for feature selection.

*   **`Feature Selector:`** This function selects (or in other words decides class of) most informative feature (in other words which provides most information gain). This is a critical helper function for decision tree generator.

*   **`Decision Tree (Class):`** This is class definition of decision tree. It has a simple constructor which includes array of childs and a few important string data.

*   **`Index Finder:`** This is a simple function which returns index of given column name. This is required for numpy operations.

*   **`Decision Tree Generator:`** This function creates decision tree for our classification model.

*   **`Predictor:`** This function makes predictions for given instance with iterating over given decision tree recursively.

*   **`Model Evaluator:`** This function calculates models accuracy, precision, recall and F1-score metrics with creating a confusion matrix.

In [2]:
def kFoldCrossValidationSplit(dataframe, k, validation_index):
    # Step-1: Splitting dataset into k folds
    folds = np.array_split(dataframe, k)
    trainFolds = [];
    
    # Step-2: Deciding indexes of train and validation parts
    for i in range(k):
        if i == validation_index:
            # Assigning validation dataset
            validationSet = folds[validation_index]
        else:
            trainFolds.append(folds[i]);
    
    # Step-3: Assigning train dataset
    trainSet = trainFolds.pop(0)   
    for i in range(len(trainFolds)):
        trainSet = np.concatenate((trainSet, trainFolds[i]), axis=0)
        
    # Step-4: Returning both of training and validation datasets
    return trainSet, validationSet

In [3]:
def discretizeContFeature(dataframe, columnIndex, rangeNumber):
    # Step-1: Finding minimum and maximum values of continous feature
    min = dataframe[0][columnIndex]
    max =  min
    
    for i in range(dataframe.shape[0]):
        if dataframe[i][columnIndex] < min:
            min = dataframe[i][columnIndex]
        elif dataframe[i][columnIndex] > max:
            max = dataframe[i][columnIndex]
            
    # Step-2: Calculating size of ranges for equal width binning
    rangeSize = math.ceil((max - min) / rangeNumber)

    # Warning! Adding an error handling for zero division case (it is necessary in this dataset)
    if(rangeSize == 0): rangeSize = 1 

    # Step-3: Replacing continous features for each row
    for i in range(dataframe.shape[0]):
        # Step-3.1: Calculating the order of the continuous value
        rangeOrder = math.ceil((dataframe[i][columnIndex] - min + 1) / rangeSize)
        
        # Step-3.2: Creating a new label
        rangeLabel = str(math.floor(min + (rangeOrder - 1) * rangeSize)) +  "-" + str(math.floor(min + (rangeOrder) * rangeSize))
        
        # Step-3.3: Updating the label of continuous value
        dataframe[i][columnIndex] = rangeLabel

In [4]:
def getGroupedRecordsByFeatureValue(numpyArray, dataColumnNames):
    # Example Output For A Mini Dataset:
    # Grouped Records: {'Age': {'18-39': 39, '39-60': 27}, 'BusinessTravel': {'Travel_Rarely': 43, 'Travel_Frequently': 19, 'Non-Travel': 4}, 'DailyRate': {'801-1500': 41, '102-801': 25}, 'Department': {'Sales': 22, 'Research & Development': 41, 'Human Resources': 3}, 'DistanceFromHome': {'1-15': 54, '15-29': 11, '29-43': 1}, 'Education': {'3-5': 43, '1-3': 20, '5-7': 3}, 'EducationField': {'Medical': 18, 'Other': 7, 'Life Sciences': 31, 'Marketing': 4, 'Human Resources': 2, 'Technical Degree': 4}, 'EmployeeCount': {'1-2': 66}, 'EmployeeNumber': {'1035-2069': 35, '1-1035': 31}, 'EnvironmentSatisfaction': {'3-5': 40, '1-3': 26}, 'Gender': {'Female': 28, 'Male': 38}, 'HourlyRate': {'30-65': 34, '65-100': 31, '100-135': 1}, 'JobInvolvement': {'3-5': 50, '1-3': 16}, 'JobLevel': {'1-3': 49, '3-5': 13, '5-7': 4}, 'JobRole': {'Sales Representative': 9, 'Research Scientist': 14, 'Sales Executive': 12, 'Healthcare Representative': 8, 'Manager': 8, 'Research Director': 1, 'Manufacturing Director': 3, 'Laboratory Technician': 11}, 'JobSatisfaction': {'3-5': 41, '1-3': 25}, 'MaritalStatus': {'Single': 26, 'Married': 28, 'Divorced': 12}, 'MonthlyIncome': {'1009-10504': 54, '10504-19999': 12}, 'MonthlyRate': {'2094-14547': 30, '14547-27000': 36}, 'NumCompaniesWorked': {'0-5': 48, '5-10': 18}, 'Over18': {'Y': 66}, 'OverTime': {'No': 50, 'Yes': 16}, 'PercentSalaryHike': {'18-25': 17, '11-18': 48, '25-32': 1}, 'PerformanceRating': {'3-4': 58, '4-5': 8}, 'RelationshipSatisfaction': {'1-3': 30, '3-5': 36}, 'StandardHours': {'80-81': 66}, 'StockOptionLevel': {'0-2': 58, '2-4': 8}, 'TotalWorkingYears': {'0-20': 53, '20-40': 13}, 'TrainingTimesLastYear': {'3-6': 32, '0-3': 31, '6-9': 3}, 'WorkLifeBalance': {'3-5': 44, '1-3': 22}, 'YearsAtCompany': {'0-20': 63, '20-40': 3}, 'YearsInCurrentRole': {'0-9': 62, '9-18': 4}, 'YearsSinceLastPromotion': {'0-8': 64, '8-16': 2}, 'YearsWithCurrManager': {'0-9': 61, '9-18': 5}, 'Attrition': {'No': 56, 'Yes': 10}}

    # Step-1: Initializing dictionary of results
    groupedRecords = {}

    # Step-2: Mapping values
    for j in range(numpyArray.shape[1]):
      # Step-2.1: Creating sub dictionaries for each of sub features
      groupedRecords[dataColumnNames[j]] = {}
      for i in range(numpyArray.shape[0]):
        # Step-2.2: If feature exists, we increasing occurence number
        if numpyArray[i][j] in groupedRecords[dataColumnNames[j]]:
          groupedRecords[dataColumnNames[j]][numpyArray[i][j]] += 1
        # Step-3.3: Otherwise, we create a instance for that feature
        else:
          groupedRecords[dataColumnNames[j]][numpyArray[i][j]] = 1;

    # Step-3: Returning the dictionary
    return groupedRecords

In [5]:
def calculateSystemEntropy(numpyArray, groupedRecords):
  # Step-1: Initializing system entropy and important values
  totalEntropy = 0
  drugA = 0
  drugB = 0

  # Step-2: Dealing with exceptions (rarely they may occur)
  # If such an exception exits, entropy will tend to be less
  try:  
    drugA = groupedRecords["Attrition"]["Yes"]
  except:
    drugA = 1
  try:  
    drugB = groupedRecords["Attrition"]["Yes"]
  except:
    drugB = 1

  # Step-3: Calculating the system entropy
  totalEntropy += -1 * (drugA / numpyArray.shape[0]) * math.log2(drugA / numpyArray.shape[0])
  totalEntropy += -1 * (drugB / numpyArray.shape[0]) * math.log2(drugB / numpyArray.shape[0])

  # Step-4: Returning the system entropy
  return totalEntropy

In [6]:
def calculateFeatureEntropy(numpyArray, groupedRecords, systemEntropy, dataColumnNames):
  # Example Output For A Mini Dataset:
  # Feature Entropies: {'Age': [0.7320666900931938, 0.3809465857053901], 'BusinessTravel': [0.5830194167347008, 0.6292492238560345, 0.8112781244591328], 'DailyRate': [0.5349436990971067, 0.7219280948873623], 'Department': [0.6840384356390417, 0.6006085754131871, -0.0], 'DistanceFromHome': [0.3809465857053901, 0.9940302114769565, -0.0], 'Education': [0.5185697317883058, 0.8112781244591328, -0.0], 'EducationField': [0.5032583347756457, 0.863120568566631, 0.6373874992221911, -0.0, -0.0, 0.8112781244591328], 'EmployeeCount': [0.6136190195993708], 'EmployeeNumber': [0.6609623351442084, 0.5547781633412736], 'EnvironmentSatisfaction': [0.5435644431995964, 0.7062740891876007], 'Gender': [0.676941869780886, 0.5617526078313282], 'HourlyRate': [0.3227569588973982, 0.7706290693639405, -0.0], 'JobInvolvement': [0.584238811642856, 0.6962122601251459], 'JobLevel': [0.5916727785823275, 0.7793498372920852, -0.0], 'JobRole': [-0.0, 0.5916727785823275, 0.9182958340544896, 0.5435644431995964, 0.5435644431995964, -0.0, -0.0, 0.6840384356390417], 'JobSatisfaction': [0.6593758812786992, 0.5293608652873644], 'MaritalStatus': [0.8403586716091171, 0.49123734182433315, -0.0], 'MonthlyIncome': [0.6051865766334206, 0.6500224216483541], 'MonthlyRate': [0.35335933502142136, 0.7642045065086203], 'NumCompaniesWorked': [0.6500224216483541, 0.5032583347756457], 'Over18': [0.6136190195993708], 'OverTime': [0.32744491915447627, 0.9886994082884974], 'PercentSalaryHike': [0.6722948170756379, 0.5993142373098092, -0.0], 'PerformanceRating': [0.6226343162547098, 0.5435644431995964], 'RelationshipSatisfaction': [0.6500224216483541, 0.5813214987637028], 'StandardHours': [0.6136190195993708], 'StockOptionLevel': [0.6631968402398287, -0.0], 'TotalWorkingYears': [0.6572729784684465, 0.39124356362925566], 'TrainingTimesLastYear': [0.5435644431995964, 0.7088356733321961, -0.0], 'WorkLifeBalance': [0.5746356978376794, 0.6840384356390417], 'YearsAtCompany': [0.631263018091612, -0.0], 'YearsInCurrentRole': [0.6373874992221911, -0.0], 'YearsSinceLastPromotion': [0.6252624052234231, -0.0], 'YearsWithCurrManager': [0.6436394131461666, -0.0]}

  # Step-1: Initializing dictionary of results
  entropies = {}

  # Step-2: Mapping values
  for j in range(numpyArray.shape[1]-1):
    # Step-2.1: Creating sub dictionaries for each of sub features
    entropies[dataColumnNames[j]] = {}
    keys = list(groupedRecords[dataColumnNames[j]].keys())
    values = list()
    valuesA = list()
    valuesB = list()

    # Step-2.2: Initializing important instances which will be used at calculation
    for i in range(len(keys)):
      values.append(groupedRecords[dataColumnNames[j]].get(keys[i]))
      valuesA.append(0)
      valuesB.append(0)
       
    # Step-2.3: Updating important instances which will be used at calculation
    for i in range(numpyArray.shape[0]):
      for q in range(len(keys)):
        if(numpyArray[i][j] == keys[q]):
          if(numpyArray[i][len(dataColumnNames)-1] == "Yes"):
            valuesA[q] += 1
          if(numpyArray[i][len(dataColumnNames)-1] == "No"):
            valuesB[q] += 1

    # Step-3: Calculating the entropies for each instance
    subList = list()
    for q in range(len(list(groupedRecords[dataColumnNames[j]].keys()))):
      val1 = valuesA[q]/(valuesA[q]+valuesB[q])
      val2 = valuesB[q]/(valuesA[q]+valuesB[q])

      if(val1==0): val1=1
      if(val2==0): val2=1
                          
      entropy = ((-1.00) * (valuesA[q]/(valuesA[q]+valuesB[q])) * math.log2(val1)) - ((valuesB[q]/(valuesA[q]+valuesB[q])) * math.log2(val2))
      subList.append(entropy)
        
      entropies[dataColumnNames[j]] = subList

  # Step-4: Returning the dictionary
  return entropies

In [7]:
def calculateGain(systemEntropy, featureEntropies, dataColumnNames, groupedRecords, numpyArray):
  # Example Output For A Mini Dataset:
  # Information Gains: {'Age': 0.23656305699608685, 'BusinessTravel': 0.21482784573831062, 'DailyRate': 0.21921858264024835, 'Department': 0.22387156535488234, 'DistanceFromHome': 0.3476344323497612, 'Education': 0.2412918111112427, 'EducationField': 0.2476469925722626, 'EmployeeCount': 0.21137068478580834, 'EmployeeNumber': 0.21390175296659164, 'EnvironmentSatisfaction': 0.21732752185697496, 'Gender': 0.21436892481767494, 'HourlyRate': 0.29675882934285036, 'JobInvolvement': 0.21360581432237408, 'JobLevel': 0.2322091886983435, 'JobRole': 0.2867408742029146, 'JobSatisfaction': 0.21486193522440986, 'MaritalStatus': 0.28553559782580984, 'MonthlyIncome': 0.21165115593086153, 'MonthlyRate': 0.24753300309801302, 'NumCompaniesWorked': 0.21499385188392733, 'Over18': 0.21137068478580834, 'OverTime': 0.3372406666527583, 'PercentSalaryHike': 0.21595795770098697, 'PerformanceRating': 0.2119396151674528, 'RelationshipSatisfaction': 0.21244051340118036, 'StandardHours': 0.21137068478580834, 'StockOptionLevel': 0.24218035993199638, 'TotalWorkingYears': 0.22011645914263397, 'TrainingTimesLastYear': 0.22850533990510086, 'WorkLifeBalance': 0.21388642728037904, 'YearsAtCompany': 0.2224204598431858, 'YearsInCurrentRole': 0.22623175057039357, 'YearsSinceLastPromotion': 0.21867464477458698, 'YearsWithCurrManager': 0.23011085284099497}

  # Step-1: Initializing dictionary of results
  gains = {}

  # Step-2: Mapping values
  for i in range(numpyArray.shape[1]-1):
    # Step-2.1: Initializing keys; and entropy variable as system entropy
    keys = list(groupedRecords[dataColumnNames[i]].keys())
    entropy = systemEntropy

    # Step-2.2: Calculating total occurence of current variable
    total = 0
    for j in range(len(keys)):
      total += groupedRecords[dataColumnNames[i]].get(keys[j])

    # Step-2.3: Updating entropy with using standart formula
    for j in range(len(keys)):
      curr = groupedRecords[dataColumnNames[i]].get(keys[j])
      entropy -= (curr/total) * featureEntropies[dataColumnNames[i]][j]
      
    # Step-2.4: Updating gain value with calculated output
    gains[dataColumnNames[i]] = entropy

  # Step-3: Returning the dictionary
  return gains

In [8]:
def decideClass(numpyArray, index, groupedRecords, value, dataColumnNames, decideindex):

  # Step-1: Initializing important variables
  keys = list(groupedRecords[dataColumnNames[index]].keys())
  values = list()
  valuesA = list() # Attrition = Yes
  valuesB = list() # Attrition = No

  for i in range(len(keys)):
    values.append(groupedRecords[dataColumnNames[index]].get(keys[i]))
    valuesA.append(0)
    valuesB.append(0)
        
  # Step-2: Calculating occurences of values of 'Attrition' column
  for i in range(numpyArray.shape[0]):
    for q in range(len(keys)):
      if(numpyArray[i][index] == keys[q]):
        if(numpyArray[i][len(dataColumnNames)-1] == "Yes"):
          valuesA[q] += 1
        if(numpyArray[i][len(dataColumnNames)-1] == "No"):
          valuesB[q] += 1
  
  # Step-3: Returning result which occurs more
  if(valuesA[decideindex] > valuesB[decideindex]): return "Yes"
  else: return "No"

In [9]:
class DecisionTree:
  # Decision tree generator
  def __init__(self):
    self.childs = [] # Childs are in array since there can be different size of childs
    self.main = ""   # Main -> Name of root (for example age)
    self.name = ""   # Name -> Name of child (for example '18-39' or '39-60')
    self.data = ""   # Data -> Name of childs data (for example 'Attrition: Yes' or another column name such as 'BusinessTravel')
    self.information = "" # Information -> Information gain value (for twigs)
    self.values = {} # Values -> Values for attrition = yes or no

In [10]:
def getIndexOfColumn(dataColumnNames,columnName):
  # Helper Function: Returns index of given column name
  for i in range(len(dataColumnNames)):
    if(dataColumnNames[i] == columnName):
      return i
  return -1

In [11]:
def createDecisionTree(numpyArray, dataColumnNames, depth, maxdepth, willPrint="false", root="root"):

  if numpyArray.shape[0] > 0 and numpyArray.shape[1] > 1 and maxdepth > depth:

    # Step-1: Initializing and calculating required variables
    groupedRecords = getGroupedRecordsByFeatureValue(numpyArray, dataColumnNames)
    systemEntropy = calculateSystemEntropy(numpyArray, groupedRecords)
    featureEntropies = calculateFeatureEntropy(numpyArray, groupedRecords, systemEntropy, dataColumnNames)
    gains = calculateGain(systemEntropy, featureEntropies, dataColumnNames, groupedRecords, numpyArray)
    mostInformative = next(iter(dict(sorted(gains.items(), key=operator.itemgetter(1), reverse=True))))

    # Step-2: Creating the current root node of decision tree
    decisionTree = DecisionTree()
    decisionTree.childs = []
    decisionTree.name = root
    decisionTree.main = mostInformative
    decisionTree.data = mostInformative
    decisionTree.information = gains[list(gains.keys())[getIndexOfColumn(dataColumnNames,mostInformative)]]
    decisionTree.values = groupedRecords
    if(willPrint == True):  print("(" + (str(depth) + ") " + str(decisionTree.data) + " {"), end=" ")

    # Step-3: Calculating values which will be used to creating child nodes of current root
    constructValues = {}
    for i in range(len(list(groupedRecords[dataColumnNames[getIndexOfColumn(dataColumnNames,mostInformative)]]))):
      keys = list(groupedRecords[dataColumnNames[getIndexOfColumn(dataColumnNames,mostInformative)]].keys())
      index = getIndexOfColumn(dataColumnNames,mostInformative)
      if(len(constructValues) == 0):
        for j in range(len(keys)):
          constructValues[keys[j]] = featureEntropies[dataColumnNames[index]][j]
      constructValues = dict(sorted(constructValues.items(), key=operator.itemgetter(1), reverse=True))

    # Step-4: Creating first child of tree (which will have value of 'Attrition' = Yes or 'Attrition' = No)
    index = getIndexOfColumn(dataColumnNames,mostInformative)
    nThChild = DecisionTree()
    nThChild.childs = []
    nThChild.main = mostInformative
    nThChild.data = decideClass(numpyArray, index, groupedRecords, list(constructValues.keys())[0], dataColumnNames, i)
    nThChild.name = list(constructValues.keys())[0]
    nThChild.information = "100000" 
    nThChild.values = {}
    decisionTree.childs.append(nThChild)
    if(willPrint == True):  print("(" + (str(depth) + ") " + str(nThChild.data) + " {"), end=" ")

    # Step-5: Updating dataset and column names with slicing for next depth recursion
    index = getIndexOfColumn(dataColumnNames,mostInformative)
    temp = np.array(numpyArray, copy=True)

    try:
      for i in range(temp.shape[0]):  
        if(temp[i][index] == list(constructValues.keys())[index]):
          numpyArray = np.delete(numpyArray, i, 0)
          i-=1    
    except:
      pass

    dataColumnNames = np.delete(dataColumnNames, index)
    numpyArray = np.delete(numpyArray, index, 1)

    # Step-6: If it is still possible to create childs, then creating them recursively
    if(dataColumnNames.size > 0):
      for i in range(1, len(constructValues)):
        nThChild = createDecisionTree(numpyArray, dataColumnNames, depth+1, maxdepth, willPrint, list(constructValues.keys())[i])
        # Checking if it is possible to create childs
        if(nThChild is not None):
          decisionTree.childs.append(nThChild)
        # If it is not possible, then deciding result (which will have value of 'Attrition' = Yes or 'Attrition' = No)
        else:
          try:
            decisionTree.data = decideClass(numpyArray, index, groupedRecords, list(constructValues.keys())[0], dataColumnNames, 0)
          except:        
            decisionTree.data = "No"
        if(willPrint == True):  print(" }", end=" ")

    # Step-7: Returning decision tree 
    return decisionTree

In [12]:
# Since it is a recursive function and since it doesn't stop with first return value, i implemented it a bit different.
predictions = list()

def makePrediction(tree, instance, dataColumnNames):
    # Step-1: Initializing predictions and important variables
    global predictions
    predictions = list()
    index = getIndexOfColumn(dataColumnNames,tree.data)

    # Step-2: Search childs recursively
    for i in range(len(tree.childs)):
      # Step-2.1: Checking whether is the child is correct one
      if(tree.childs[i].name == instance[index]):
        # Step-2.2: Checking whether it is a leaf node, if child is correct one
        if((tree.childs[i].data == "Yes") or (tree.childs[i].data == "No")):
          predictions.append(tree.childs[i].data)
        # Step-3.3: Making search recursively, if it is not leaf node
        else:
          makePrediction(tree.childs[i], instance, dataColumnNames)

    # Step-3.1: Select and return first prediction
    try: 
      prediction = predictions[0]
    # Step-3.2: If any exception will occur, return most frequent one
    except: 
      prediction = "No"

    # Step-4: Returining predicted value for attrition (Yes or No)
    return prediction

In [13]:
def modelEvaluation(decisionTree, validationData, columnNames, doPrint=False):
    # Step-1: Creating a 2x2 confusion matrix
    TP = 0
    TN = 0
    FP = 0
    FN = 0

    # Step-2: Updating values of confusion matrix with using predictions and actual values
    for i in range(validationData.shape[0]):
        predictedClass = makePrediction(decisionTree, validationData.iloc[i], columnNames)

        if(doPrint):
          if(predictedClass != validationData.iloc[i]["Attrition"]):
            print("------- " + str(i) + "th Sample -------")
            print("Original: " + str(validationData.iloc[i]["Attrition"]) + " - Predicted: " + str(predictedClass))

        if predictedClass == "Yes" and validationData.iloc[i]["Attrition"] == "Yes":
            TP += 1
        elif predictedClass == "Yes" and validationData.iloc[i]["Attrition"] == "No":
            FP += 1
        elif predictedClass == "No" and validationData.iloc[i]["Attrition"] == "Yes":
            FN += 1
        elif predictedClass == "No" and validationData.iloc[i]["Attrition"] == "No":
            TN += 1

    # Step-3: Calculating model evaluation metrics
    accuracy = (TP + TN) / (TP + FP + FN + TN + 0.001)
    precision = (TP) / (TP + FP + 0.001)
    recall = (TP) / (TP + FN + 0.001)
    f1score = (2 * (recall * precision)) / (recall + precision + 0.001)
    
    # Step-4: Returning model evaluation metrics
    return accuracy, precision, recall, f1score

##**1.3 - Dataset Preparation**

In [15]:
# Uploading and reading dataset
df_name = "WA_Fn-UseC_-HR-Employee-Attrition.csv"
df = pd.read_csv(df_name)

# Storing names of columns
columnNames = np.array(df.columns)

df.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0


In [16]:
# Storing indexes of continuous features
indexesOfContFeatures = list()
count = 0

for i in range(df.dtypes.size):
  if(df.dtypes[i] == "int64"):
    indexesOfContFeatures.append(i)
    count += 1

print("Number of Continous Features: " + str(count))

# It is same with information at kaggle, there are 26 features (others are string and boolean)

Number of Continous Features: 26


In [17]:
# Converting dataframe to numpy array
df = np.array(df)

# Rearranging target column to the last column
df = df[:,[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,1]]

# Updating indexes of continuous features
for i in range(len(indexesOfContFeatures)):
  if (indexesOfContFeatures[i] != 0): indexesOfContFeatures[i] -= 1

# Updating column names
newColumnNames = list()
for i in range(df.shape[1]):
  if(i == 0): newColumnNames.append(columnNames[i])
  elif(i == 34): newColumnNames.append(columnNames[1])
  else: newColumnNames.append(columnNames[i+1])

columnNames = np.array(newColumnNames)

# Shuffling the array in order to reduce risk
np.random.shuffle(df)

print("Dataframe Before Discretion of Continous Values:")
print(df[:,:])

# Discretizing continuous features
for i in range(len(indexesOfContFeatures)):
  discretizeContFeature(df, indexesOfContFeatures[i], 2)

print("")
print("Dataframe Before Discretion of Continous Values:")
print(df[:,:])

Dataframe Before Discretion of Continous Values:
[[39 'Non-Travel' 1485 ... 0 5 'No']
 [18 'Non-Travel' 247 ... 0 0 'Yes']
 [23 'Travel_Rarely' 310 ... 0 2 'No']
 ...
 [55 'Travel_Rarely' 189 ... 1 4 'No']
 [30 'Travel_Rarely' 853 ... 8 9 'No']
 [44 'Travel_Rarely' 200 ... 13 17 'No']]

Dataframe Before Discretion of Continous Values:
[['39-60' 'Non-Travel' '801-1500' ... '0-8' '0-9' 'No']
 ['18-39' 'Non-Travel' '102-801' ... '0-8' '0-9' 'Yes']
 ['18-39' 'Travel_Rarely' '102-801' ... '0-8' '0-9' 'No']
 ...
 ['39-60' 'Travel_Rarely' '102-801' ... '0-8' '0-9' 'No']
 ['18-39' 'Travel_Rarely' '801-1500' ... '8-16' '9-18' 'No']
 ['39-60' 'Travel_Rarely' '102-801' ... '8-16' '9-18' 'No']]


##**1.4 - Executing & Evaluating Model**

###**1.4.1 - Executing & Evaluating Model With Maximum Tree Depth**

In [14]:
def executemodel(df,columnNames,foldNumber,maxDepth,willPrint=False):
  print("--- CLASSIFICATION REPORT ---")

  # Repeating decision tree and model evaluation algorithms for each of folds 
  for k in range(foldNumber):
    print("Current Fold: " + str(k+1))

    # Step-1: Splitting dataset to train and validation datasets
    trainSet, validationSet = kFoldCrossValidationSplit(df, foldNumber, k)  

    # Step-2: Creating the decision tree with using training dataset
    if(willPrint): print("Current Tree Rule: ", end="")
    decisionTree = createDecisionTree(trainSet, columnNames, 0, maxDepth, willPrint)

    # Step-3: Evaulating the model with using validation dataset 
    validationSet = pd.DataFrame(validationSet, columns = columnNames)     
    accuracy, precision, recall, f1score = modelEvaluation(decisionTree, validationSet, columnNames)
          
    # Step-4: Printing the results for each fold
    print("")
    print("Accuracy: ", str("{:.2f}".format(accuracy)))
    print("Precision: ", str("{:.2f}".format(precision)))
    print("Recall: ", str("{:.2f}".format(recall)))
    print("F1 Score: ", str("{:.2f}".format(f1score)))
    print(" ")

In [ ]:
executemodel(df[:,:],columnNames,5,34,False) 
# 34 = max height of tree = number of features in dataset

--- CLASSIFICATION REPORT ---
Current Fold: 1

Accuracy:  0.52
Precision:  0.05
Recall:  0.11
F1 Score:  0.07
 
Current Fold: 2

Accuracy:  0.81
Precision:  0.00
Recall:  0.00
F1 Score:  0.00
 
Current Fold: 3

Accuracy:  0.51
Precision:  0.06
Recall:  0.16
F1 Score:  0.09
 
Current Fold: 4

Accuracy:  0.78
Precision:  0.00
Recall:  0.00
F1 Score:  0.00
 
Current Fold: 5

Accuracy:  0.83
Precision:  0.14
Recall:  0.02
F1 Score:  0.04
 


 **Average Accuracy:** ~%70

###**1.4.2 - Executing & Evaluating Model With Smaller Tree Depth**

Tree depth was 34, it's execution time was more than 6 hours. Therefore it includes quite many rules. If i try to write all of them to the console, program crashes since there are to many things to write. Thats why now we will try to write rules with a smaller depth of tree.

**(Note:** One block above, if you change False as True, then it will write tree rules properly but my computer crashes due to excessive process load if i do it.**)**

In [ ]:
executemodel(df[:,:],columnNames,5,7,True) # 7 = max height of tree

# If i try to write rules with depth=34, then program crashes since it will print thousands of values
# Therefore, with limiting height and fold; i can show you that program works properly

--- CLASSIFICATION REPORT ---
Current Fold: 1
Current Tree Rule: (0) JobRole { (0) No { (1) OverTime { (1) No { (2) MaritalStatus { (2) No { (3) BusinessTravel { (3) No { (4) TotalWorkingYears { (4) Yes { (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  } (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  }  } (4) TotalWorkingYears { (4) Yes { (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  } (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  }  }  } (3) BusinessTravel { (3) No { (4) TotalWorkingYears { (4) Yes { (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  } (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  }  } (4) TotalWorkingYears { (4) Yes { (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  } (5) MonthlyIncome { (5) No { (6) JobInvolvement { (6) No {  }  }  }  }  }  }  } (1) OverTime { (1) No { (2) MaritalStatus { (2) No { (3) BusinessTravel { (3) No { (4) TotalWor

 **Average Accuracy:** ~%58

##**1.5 - Error Analysis for Classification**

###**1.5.1 - General Performance Analysis**

**Accuracy:** According to the accuracy value, our decision tree model predicts %70 of samples correctly.

**Precision & Recall:** According to the precision and recall values, model can't find true positives well.

**F1-Score:** According to F1-score, decision tree with maximum depth is not a great option to find true positives (rare group in the current dataset).

**General Performance:** Since accuracy is sufficient while precision, recall and F1-scores are quite bad; it shows that model predicts quite less true positives but quite many true negatives. (in other words **overfitting**)

---

**Note:** Since each depth of the decision tree has a 'yes' or 'no' value, increasing the maximum depth of the decision tree loses its effect on performance metrics after a certain value (approximately 10) and causes a serious loss of time in training the model.

---

###**1.5.2 - Factors That Make Hard To Classify**

**1. Maximized Tree Depth:** By default, the decision tree model is allowed to grow to its full depth. If model has full depth, it means that model has ability to classification in specific conditions. Therefore, if a tree that is too large, it has risk of overfitting the training data and poorly generalizing to new samples. As you can see, after we reduced maximum tree depth to 7 in 1.4.2 part; overfitting reduced.

**2. Data Distribution:** Model predicts (mostly) only negative (TN) values since %85 of samples' "Attrition" value is "No". Since decision trees are based on power of majority and "No" value is majority; it is not surprising that the model mostly returns negative values.

**3. Type of Discretization Process:** Equal depth (frequeny) binning usually tends to give better results. But I used equal width binning because it is easier to implement and it is stated on the PDF. 

###**1.5.3 - Misclassified Sample Examples**

In [ ]:
# Step-1: Splitting dataset to train and validation datasets
trainSet, validationSet = kFoldCrossValidationSplit(df[:500], 5, 4)

# Step-2: Creating the decision tree with using training dataset
print("Current Tree Rule: ", end="")
decisionTree = createDecisionTree(trainSet, columnNames, 0, 15, True)
print("")

# Step-3: Evaulating the model with using validation dataset 
validationSet = pd.DataFrame(validationSet, columns = columnNames)
accuracy, precision, recall, f1score = modelEvaluation(decisionTree, validationSet, columnNames, True)
print("Accuracy: ", str("{:.2f}".format(accuracy)))

Current Tree Rule: (0) OverTime { (0) No { (1) JobRole { (1) No { (2) YearsInCurrentRole { (2) No { (3) MaritalStatus { (3) No { (4) EducationField { (4) No { (5) YearsWithCurrManager { (5) No { (6) Age { (6) No { (7) JobLevel { (7) No { (8) TotalWorkingYears { (8) No { (9) MonthlyIncome { (9) No { (10) DistanceFromHome { (10) No { (11) PerformanceRating { (11) No { (12) WorkLifeBalance { (12) No { (13) YearsAtCompany { (13) No { (14) Education { (14) No {  }  }  }  }  }  } (11) PerformanceRating { (11) No { (12) WorkLifeBalance { (12) No { (13) YearsAtCompany { (13) No { (14) Education { (14) No {  }  }  }  }  }  }  }  }  } (8) TotalWorkingYears { (8) No { (9) MonthlyIncome { (9) No { (10) DistanceFromHome { (10) No { (11) PerformanceRating { (11) No { (12) WorkLifeBalance { (12) No { (13) YearsAtCompany { (13) No { (14) Education { (14) No {  }  }  }  }  }  } (11) PerformanceRating { (11) No { (12) WorkLifeBalance { (12) No { (13) YearsAtCompany { (13) No { (14) Education { (14) No {

The code block above shows misclassified samples in a decision tree model, where 500 samples are used and maximum depth of tree is 15. In that example;

**1-** If original class = yes and predicted class = no; it means that model misclassified it due to majority of "no" in dataset (%80).  

**2-** If original class = no and predicted class = yes; it means that model misclassified it due to majority of "yes" in some features (which is occured in splitting just by chance) But it is a rare situtation.

As you can see, **almost all of our misclassified samples are like in option 1**.

#**Part-2: Pruning Decision Tree**

##**2.1 - Implementing Required Functions**

*   **`Triple Cross Validation Splitter:`** This function divides dataframe into k folds and returns one fold as validation dataset, one fold as test set and others as training dataset.

*   **`Twig Information Finder:`** This function checks all children nodes to decide whether it is twig. Then returns informations of all twigs. This is a critical helper function for deciding which twigs will be pruned.

*   **`Leaf Count Finder:`** This function finds number of leaf nodes in the whole decision tree. This is a critical helper function for pruning and pruning evaluation processes.

*   **`Twig Pruner:`** This function prunes selected twigs and reduces overfitting risk of decision tree model.

*   **`Model Optimizer:`** This is main helper function for twig pruner. This function tests optimized model with validation set and determines that whether pruning process will continue.

*   **`Rule Writer:`** This function writes all of decision tree
model’s rules (root-to-leaf paths) with left to right order.

In [18]:
def tripleCrossValidationSplit(dataset):
  # Step-1: Splitting data into 5 folds
  folds = np.array_split(dataset, 5)
    
  # Step-2: Assigning first 3 folds as training set
  trainingFolds = [];
  trainingFolds.append(folds[0]);
  trainingFolds.append(folds[1]);
  trainingFolds.append(folds[2]);
    
  # Step-3: Assigning first training data fold to trainingSet to concatenate later
  trainingSet = trainingFolds.pop(0)
    
  # Step-4: Concatenating remaining training data folds to create training dataset
  for i in range(len(trainingFolds)):
    trainingSet = np.concatenate((trainingSet, trainingFolds[i]), axis=0)
        
  # Step-5: Creating validation and test datasets
  validationSet = np.array(folds[3])
  testSet = np.array(folds[4])

  # Step-6: Returning splitted datasets
  return trainingSet, validationSet, testSet

In [19]:
twigInformations = list()

def findTwigInformations(tree, depth=0):
  # Step-1: Initializing the system
  global twigInformations
  if(depth == 0):
    twigInformations = list()

  # Step-2: If it is a twig, append its information gain
  # Leaves has 0 tree information in current implementation
  if(tree.information != "0"):
  #if(len(tree.childs) != 0):
    twigInformations.append(str(tree.information))

  # Step-3: Apply same process for each of child recursively
  for i in range(len(tree.childs)):
    findTwigInformations(tree.childs[i], depth+1)

  # Step-4: Return twigs' information gain values
  return twigInformations

In [20]:
leafCount = 0

def findLeafCount(tree, depth=0):
  # Step-1: Initializing the system
  global leafCount
  if(depth == 0):
    leafCount = 0

  # Step-2: Checking whether current node is a twig
  if(len(tree.childs) == 0):
    if(tree.data == "Yes" or tree.data == "No"):
      leafCount += 1

  # Step-3: If it is not a twig, making a recursive search in all childs
  else:
    for i in range(len(tree.childs)):
      findLeafCount(tree.childs[i], depth+1)

  # Step-4: Returning the leaf node count
  return leafCount

In [21]:
updTree = DecisionTree()
prunedFeatures = list()

def pruneTwig(tree, leastGain, depth, trainSet, columnNames, accuracy):
  
  global updTree
  global prunedFeatures

  # Step-1: Storing tree at initial step since it is a recursive function
  if(depth == 0):
    updTree = tree
    
  # Step-2: Checking current node and if has least gain, pruning it
  if(str(tree.information) == str(leastGain)):

    # Step-2.1: Checking whether it is a twig
    if(len(tree.childs) > 0):

      # Step-2.2: Removing all child nodes of the twig
      for i in range(len(tree.childs)):
        if ("(" + str(tree.childs[i].main) + " -> " + str(tree.childs[i].name) + ")") not in prunedFeatures:
          prunedFeatures.append("(" + str(tree.childs[i].main) + " -> " + str(tree.childs[i].name) + ")")

      tree.childs = []   

      # Step-2.3: Deciding class of leaf
      # Step-2.3.1: Initializing important variables
      index = getIndexOfColumn(columnNames, tree.main)
      keys = list(tree.values[columnNames[index]].keys())
      values = list()
      valuesA = list() # Attrition = Yes
      valuesB = list() # Attrition = No

      for i in range(len(keys)):
        values.append(tree.values[columnNames[index]].get(keys[i]))
        valuesA.append(0)
        valuesB.append(0)

      # Step-2.3.2: Calculating occurences of values of 'Attrition' column
      for i in range(trainSet.shape[0]):
        for q in range(len(keys)):
          if(trainSet[i][index] == keys[q]):
            if(trainSet[i][len(columnNames)-1] == "Yes"):
              valuesA[q] += 1
            if(trainSet[i][len(columnNames)-1] == "No"):
              valuesB[q] += 1

      # Step-2.3.3: Returning result which occurs more
      yes = 0
      no = 0
      for i in range(len(valuesA)):
        yes += valuesB[i]
        no += valuesA[i]
      
      # Step-2.4: Relabeling twig as a leaf
      tree.information = "0"
      if(yes > no): tree.data = "Yes"
      else: tree.data = "No"

  # Step-3: Appyling same process for children recursively
  else:
    for i in range(len(tree.childs)):
      pruneTwig(tree.childs[i], leastGain, depth+1, trainSet, columnNames, accuracy)

  # Step-4: Finally returning the pruned tree
  return tree

In [22]:
def optimizeModel(decisionTree, trainSet, validationSet, columnNames, accuracy, twigGainss=[]):

  # Step-1: Finding twig with the least information gain value
  leastGain = ""
  twigGains = findTwigInformations(decisionTree)

  if(len(twigGains) > 0):
    twigGains.sort()
    leastGain = twigGains[0]

    # Step-2: Pruning the tree
    prunedTree = pruneTwig(decisionTree, leastGain, 0, trainSet, columnNames, accuracy)

    # Step-3: Evaulating the model with using validation dataset 
    validationSet = pd.DataFrame(validationSet, columns = columnNames)
    newAccuracy, newPrecision, newNecall, newF1Score = modelEvaluation(prunedTree, validationSet, columnNames)

    # Step-4: Continueing to optimize until thats enough
    if((newAccuracy > accuracy) and (twigGainss != twigGains)):
      optimizeModel(prunedTree, trainSet, validationSet, columnNames, accuracy, twigGains)
    
    # Step-5: Finally returning the outputs
    return prunedTree, newAccuracy, newPrecision, newNecall, newF1Score

  # Step-6: Finally returning the outputs
  validationSet = pd.DataFrame(validationSet, columns = columnNames)
  newAccuracy, newPrecision, newNecall, newF1Score = modelEvaluation(decisionTree, validationSet, columnNames)
  return decisionTree, newAccuracy, newPrecision, newNecall, newF1Score

In [23]:
treeRule = ""

def ruleWriter(tree, depth):
  global treeRule
  if depth == 0: treeRule = ""
  treeRule += ("(" + (str(depth) + ") " + str(tree.data) + " {"))
  for i in range(len(tree.childs)):
    ruleWriter(tree.childs[i], depth+1)

##**2.2 - Executing & Evaluating Pruning**


In [ ]:
# ------------------------ Original (Pre-Pruned) Model -------------------------
# Step-1: Splitting dataset to train and validation datasets
trainSet, validationSet, testSet = tripleCrossValidationSplit(df[:])

# Step-2: Creating the decision tree with using training dataset
print("------- ORIGINAL (PRE-PRUNED) TREE EVALUATION -------")
decisionTree = createDecisionTree(trainSet, columnNames, 0, 34, False)
print("Current Tree Rule Will Be Written At: original.txt")
ruleWriter(decisionTree, 0)
file_object = open('original.txt', 'a')
file_object.write(treeRule)
print("")

# Step-3: Evaulating the model with using validation dataset 
validationSet = pd.DataFrame(validationSet, columns = columnNames)
testSet = pd.DataFrame(testSet, columns = columnNames)
accuracy, precision, recall, f1score = modelEvaluation(decisionTree, testSet, columnNames)
print("Accuracy Before: ", str("{:.2f}".format(accuracy)))
print("Leaf Count Before: " + str(findLeafCount(decisionTree)))
print("Twig Count Before: " + str(len(findTwigInformations(decisionTree))))
print("")

# ------------------------- Post-Pruned Model --------------------------
# Step-1: Pruning tree recursively
prunedTree, newAccuracy, newNPrecision, newNecall, newF1Score = optimizeModel(decisionTree, trainSet, validationSet, columnNames, accuracy)

lastAccuracy, lastPrecision, lastRecall, lastF1score = modelEvaluation(prunedTree, testSet, columnNames)

# Step-2: Evaulating the model with using validation dataset 
print("------- POST-PRUNED TREE EVALUATION -------")
print("Current Tree Rule Will Be Written At: pruned.txt")
ruleWriter(prunedTree, 0)
file_object2 = open('pruned.txt', 'a')
file_object2.write(treeRule)
print("")
print("Accuracy After: ", str("{:.2f}".format(lastAccuracy)))
print("Leaf Count After: " + str(findLeafCount(prunedTree)))
print("Twig Count After: " + str(len(findTwigInformations(prunedTree))))
print("")

print("Pruned Features (Without Including Quite Many Duplicates):")
for i in range(len(prunedFeatures)):
  print(str(i) + "- " + str(prunedFeatures[i]))

------- ORIGINAL (PRE-PRUNED) TREE EVALUATION -------
Current Tree Rule Will Be Written At: original.txt

Accuracy Before:  0.84
Leaf Count Before: 145897
Twig Count Before: 291794

------- POST-PRUNED TREE EVALUATION -------
Current Tree Rule Will Be Written At: pruned.txt

Accuracy After:  0.84
Leaf Count After: 125417
Twig Count After: 230354

Pruned Features (Without Including Quite Many Duplicates):
0- (MaritalStatus -> Single)
1- (Education -> 3-5)
2- (PercentSalaryHike -> 1-3)
3- (YearsInCurrentRole -> 0-9)
4- (Education -> 9-18)
5- (NumCompaniesWorked -> 5-10)
6- (YearsInCurrentRole -> 0-5)
7- (MonthlyIncome -> 1009-10504)
8- (NumCompaniesWorked -> 10504-19999)
9- (HourlyRate -> 65-100)
10- (MonthlyIncome -> 30-65)
11- (MonthlyIncome -> 100-135)
12- (PerformanceRating -> 4-5)
13- (HourlyRate -> 3-4)
14- (WorkLifeBalance -> 1-3)
15- (PerformanceRating -> 3-5)
16- (MonthlyRate -> 14547-27000)
17- (WorkLifeBalance -> 2094-14547)
18- (RelationshipSatisfaction -> 3-5)
19- (MonthlyRa

In [24]:
print("--- ORIGINAL DECISION TREE ---")
file = open('original.txt') # 4.31 MB
for line in file:
  print(line)

--- ORIGINAL DECISION TREE ---
(0) JobRole {(1) No {(1) OverTime {(2) No {(2) EducationField {(3) No {(3) MaritalStatus {(4) No {(4) BusinessTravel {(5) No {(5) Department {(6) No {(6) Age {(7) No {(7) TrainingTimesLastYear {(8) No {(8) DistanceFromHome {(9) No {(9) JobInvolvement {(10) No {(10) JobLevel {(11) No {(11) YearsSinceLastPromotion {(12) No {(12) StockOptionLevel {(13) No {(13) EmployeeNumber {(14) No {(14) EnvironmentSatisfaction {(15) No {(15) TotalWorkingYears {(16) No {(16) JobSatisfaction {(17) No {(17) DailyRate {(18) No {(18) YearsWithCurrManager {(19) No {(19) YearsAtCompany {(20) No {(20) Gender {(21) No {(21) RelationshipSatisfaction {(22) No {(22) MonthlyRate {(23) No {(23) WorkLifeBalance {(24) No {(24) PerformanceRating {(25) No {(25) HourlyRate {(26) No {(26) MonthlyIncome {(27) No {(27) NumCompaniesWorked {(28) No {(28) YearsInCurrentRole {(29) No {(29) Education {(30) No {(30) PercentSalaryHike {(31) No {(31) EmployeeCount {(32) No {(26) MonthlyIncome {(27) N

In [25]:
print("--- PRUNED DECISION TREE ---")
file = open('pruned.txt') # 3.55 MB
for line in file:
  print(line)

--- PRUNED DECISION TREE ---
(0) JobRole {(1) No {(1) OverTime {(2) No {(2) EducationField {(3) No {(3) MaritalStatus {(4) No {(4) BusinessTravel {(5) No {(5) Department {(6) No {(6) Age {(7) No {(7) TrainingTimesLastYear {(8) No {(8) DistanceFromHome {(9) No {(9) JobInvolvement {(10) No {(10) JobLevel {(11) No {(11) YearsSinceLastPromotion {(12) No {(12) StockOptionLevel {(13) No {(13) EmployeeNumber {(14) No {(14) EnvironmentSatisfaction {(15) No {(15) TotalWorkingYears {(16) No {(16) JobSatisfaction {(17) No {(17) DailyRate {(18) No {(18) YearsWithCurrManager {(19) No {(19) YearsAtCompany {(20) No {(20) Gender {(21) No {(21) RelationshipSatisfaction {(22) No {(22) MonthlyRate {(23) No {(23) WorkLifeBalance {(24) No {(24) PerformanceRating {(25) No {(25) HourlyRate {(26) No {(26) MonthlyIncome {(27) No {(27) NumCompaniesWorked {(28) No {(28) YearsInCurrentRole {(29) No {(29) Yes {(26) MonthlyIncome {(27) No {(27) NumCompaniesWorked {(28) No {(28) YearsInCurrentRole {(29) No {(29) Yes

##**2.3 - Error Analysis for Pruning**

A decision tree which is too large has higher risk of overfitting the training set and poorly generalizing to new samples. On the other hand, a small decision tree might not use important structural information about the instances. Therefore it is important to tell when a tree algorithm should stop. Thats why we are pruning the decision tree. Pruning reduces the complexity of the final tree and therefore prevents overfitting. On the other hand pruned trees are shorter, simpler and easier to explain.

As it can be seen above, the children of the node whose children are leaf and whose has least information gain removed and the most crowded class (usually "No" value) is returned recursively. When we try this experince couple of times, even if it generally give the same accuracy, sometimes it gives better results. This may be occured because of randomization and since bigger trees are tend to overfitting. However, this may continue to remove until the accuracy become better.

Note: If less than %80 of dataset didn't have a "no attrition" value, pruning could work better. Because we always select most crowded one, and in at least %90 of it is "no". Final accuracy is quite close to "no" percentage in whole dataset.